For introduction and problem statement, please refer to notebook 1

## Content 

**Notebook 1: 1_cellphones_reviews_data_cleaning_and_eda**
- Data Import and Cleaning
- Exploratory Data Analysis
- Text Data Pre-processing

**Notebook 2: 2_cellphones_reviews_topic modelling**
- Data Import
- Topic Modelling with Gensim

**Notebook 3: 3_cellphones_reviews_topic_analysis_and_visualizations**
- [Findings and Analysis of Topic Modelling](#Findings-and-Analysis-of-Topic-Modelling)

**Notebook4**
- Sentiment Analysis (Logistic Regression Classifier, Vader and BERT) 
- Recommendation and Conclusion 
- Future Steps


In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import spacy
from nltk import tokenize
from nltk.corpus import stopwords 
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [3]:
new_reviews = pd.read_csv('../data/cleaned_combined_data.csv',na_filter=False)

In [4]:
#iphone_xs = new_reviews[new_reviews['asin'] == 'B07RT1X4FJ']

In [5]:
#iphone_xs['url'][63202]

In [6]:
new_words = {
    'new': 3.0
}

analyser.lexicon.update(new_words)

In [7]:
stop_words = stopwords.words('english')

In [8]:
len(stop_words)

179

In [9]:
negation_words = ['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
"hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 
'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
'wouldn', "wouldn't","not","no",'don',"don't"]

for word in negation_words:
    stop_words.remove(word)

len(stop_words)

139

In [10]:
def sentences_with_keywords (reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    texts = tokenize.sent_tokenize(reviews)
    for sentence in texts:
        sentence = sentence.lower()
        for word in list_of_keywords:
            if word in sentence:
                summary.add(sentence)
                
    return list(summary)

In [11]:
def summarise_reviews (reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    texts = tokenize.sent_tokenize(reviews)
    for sentence in texts:
        sentence = sentence.lower()
        for word in list_of_keywords:
            if word in sentence:
                # Remove HTML.
                post_text = BeautifulSoup(sentence).get_text()

                # Remove non-letters.
                letters_only = ' '.join(re.findall(r"[A-z’]+",post_text))

                # Convert to lower case, split into individual words.
                words = letters_only.lower().split()

                #convert the stopwords to a set.
                stops = set(stop_words)

                # Remove stopwords.
                meaningful_words = [w for w in words if w not in stops]

                # Stemming 
                #p_stemmer = PorterStemmer()
                #meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

                #Lemmatize
                lemmatizer = WordNetLemmatizer()
                meaningful_words = [lemmatizer.lemmatize(word) for word in meaningful_words]

                cleaned_sentence = (" ".join(meaningful_words))
                
                summary.add(cleaned_sentence)
                
    return list(summary)

In [12]:
def features_and_sentiments (summarised_reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    
    for cleaned_sentence in summarised_reviews:
        
        for word in list_of_keywords:
            if word in cleaned_sentence:
                score = analyser.polarity_scores(cleaned_sentence)
                compound = score['compound']

                if compound >= 0.075:
                    sentiment_score = 5
                elif compound >= 0.05:
                    sentiment_score = 4
                elif compound <= -0.075:
                    sentiment_score = 1
                elif compound <= -0.05:
                    sentiment_score = 2
                else:
                    sentiment_score = 3

                summary.add((sentiment_score,word))
    return list(summary)

In [13]:
new_reviews['summary'] = new_reviews['reviews'].apply(summarise_reviews)

In [14]:
new_reviews['sentences_with_keywords'] = new_reviews['reviews'].apply(sentences_with_keywords)

In [15]:
new_reviews['features_and_sentiments'] = new_reviews['summary'].apply(features_and_sentiments)

In [16]:
pd.set_option('display.max_colwidth',None)
new_reviews[['reviews','summary']].sample(2)

,reviews,summary
41176,Fast Very sleek and am loving it so far.,[]
50764,Phone or computer? Great phone for what I need day to day,[]


In [17]:
new_reviews["filter summary"] = new_reviews['summary'].apply(lambda x: x != [])

In [18]:
new_reviews = new_reviews[new_reviews["filter summary"] == True]

In [19]:
new_reviews.reset_index(inplace=True,drop=True)

In [20]:
new_reviews.columns

Index(['asin', 'name', 'rating', 'date', 'verified', 'review_title', 'body',
       'helpfulVotes', 'brand', 'item_title', 'url', 'image', 'reviewUrl',
       'totalReviews', 'price', 'originalPrice', 'reviews', 'word_count',
       'cleaned_reviews', 'multi_class_sentiment', 'tokens', 'summary',
       'sentences_with_keywords', 'features_and_sentiments', 'filter summary'],
      dtype='object')

In [21]:
all_features = {'camera':[],'battery':[],'fingerprint':[],'screen':[],'charger':[]}
for feature in features:
    if feature[1] =='battery':
        all_features ['battery'].append(feature[0])
    elif feature[1]  == 'camera':
        all_features ['camera'].append(feature[0])
    elif feature[1]  == 'charger':
        all_features ['charger'].append(feature[0])
    elif feature[1] == 'screen':
        all_features ['screen'].append(feature[0])
    elif feature[1] == 'fingerprint':
        all_features ['fingerprint'].append(feature[0])
        

all_features

NameError: name 'features' is not defined

In [65]:
all_features_mean = {key:np.mean(value) for key,value in all_features.items()}
all_features_mean

/Users/elisenerissa/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/elisenerissa/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'camera': 4.0,
 'battery': 4.0,
 'fingerprint': nan,
 'screen': 5.0,
 'charger': nan}

In [79]:
def mean_ratings (feature_ratings):
    all_features = {'camera':[],'battery':[],'fingerprint':[],'screen':[],'charger':[]}
    for feature in feature_ratings:
        if feature[1] =='battery':
            all_features ['battery'].append(feature[0])
        elif feature[1]  == 'camera':
            all_features ['camera'].append(feature[0])
        elif feature[1]  == 'charger':
            all_features ['charger'].append(feature[0])
        elif feature[1] == 'screen':
            all_features ['screen'].append(feature[0])
        elif feature[1] == 'fingerprint':
            all_features ['fingerprint'].append(feature[0])
    all_features_mean = {key:np.mean(value) for key,value in all_features.items()}
    
    return all_features_mean
    

In [80]:
new_reviews['vaders_ratings'] = new_reviews['features_and_sentiments'].apply(mean_ratings)

In [97]:
new_reviews.loc[0,'vaders_ratings']

{'camera': nan,
 'battery': 5.0,
 'fingerprint': nan,
 'screen': 5.0,
 'charger': nan}

In [52]:
unique_asins = new_reviews['asin'].unique()

In [91]:
for product in unique_asins:
    for idx in new_reviews.index:
        if new_reviews.loc[idx,'asin'] == product:
            

## Mean ratings by features of each unique product

In [51]:
new_reviews.reset_index(inplace=True,drop=True)

In [53]:
new_reviews.loc[1,'features_and_sentiments']

[(3, 'ringtones')]

In [98]:
all_products = {}

#for cell in new_review['features_and_sentiments']: 
all_features=set()

for product in unique_asins:
    all_products[product] = {'camera':[],'battery':[],'fingerprint':[],'screen':[],'charger':[]}
    for idx in new_reviews.index:
        if new_reviews.loc[idx,'asin'] == product:
            for feature in new_reviews.loc[idx,'features_and_sentiments']:
                all_features.add(feature[1])
                if feature[1] =='battery':
                    all_products[product]['battery'].append(feature[0])
                elif feature[1]  == 'camera':
                    all_products[product]['camera'].append(feature[0])
                elif feature[1]  == 'charger':
                    all_products[product]['charger'].append(feature[0])
                elif feature[1] == 'screen':
                    all_products[product]['screen'].append(feature[0])
                elif feature[1] == 'fingerprint':
                    all_products[product]['fingerprint'].append(feature[0])
        

In [99]:
all_products

{'B0000SX2UC': {'camera': [5, 3, 1],
  'battery': [5, 5, 5, 3, 3, 3, 1],
  'fingerprint': [],
  'screen': [5, 5, 3, 1, 5],
  'charger': [5]},
 'B000SKTZ0S': {'camera': [1, 5],
  'battery': [5, 1, 5, 1, 5, 1, 5, 3, 5, 5, 1, 3, 5, 1, 3, 5],
  'fingerprint': [],
  'screen': [5, 5, 5, 5],
  'charger': [3]},
 'B001AO4OUC': {'camera': [2, 3, 5, 5],
  'battery': [1, 3, 1, 3],
  'fingerprint': [],
  'screen': [1, 5, 1, 3, 1, 3],
  'charger': [3]},
 'B001DCJAJG': {'camera': [],
  'battery': [5],
  'fingerprint': [],
  'screen': [],
  'charger': [5]},
 'B001GQ3DJM': {'camera': [],
  'battery': [1, 5],
  'fingerprint': [],
  'screen': [5],
  'charger': [3]},
 'B0029F2O3A': {'camera': [5, 5, 4, 5, 3, 5, 5, 5, 5, 5],
  'battery': [5, 3, 1, 5, 1, 5, 5, 5, 5],
  'fingerprint': [],
  'screen': [3, 5, 1, 1, 5, 1, 5, 5, 5, 3, 5, 5, 5, 3, 5],
  'charger': []},
 'B002AS9WEA': {'camera': [5, 5],
  'battery': [5, 5, 5],
  'fingerprint': [],
  'screen': [5],
  'charger': [5]},
 'B002UHS0UI': {'camera': [5, 3

In [84]:
all_products

{'B0000SX2UC': {'camera': 3.0,
  'battery': 3.6,
  'fingerprint': nan,
  'screen': 3.8,
  'charger': 5.0},
 'B000SKTZ0S': {'camera': 3.0,
  'battery': 3.4,
  'fingerprint': nan,
  'screen': 5.0,
  'charger': 3.0},
 'B001AO4OUC': {'camera': 3.8,
  'battery': 2.0,
  'fingerprint': nan,
  'screen': 2.3,
  'charger': 3.0},
 'B001DCJAJG': {'camera': nan,
  'battery': 5.0,
  'fingerprint': nan,
  'screen': nan,
  'charger': 5.0},
 'B001GQ3DJM': {'camera': nan,
  'battery': 3.0,
  'fingerprint': nan,
  'screen': 5.0,
  'charger': 3.0},
 'B0029F2O3A': {'camera': 4.7,
  'battery': 3.9,
  'fingerprint': nan,
  'screen': 3.8,
  'charger': nan},
 'B002AS9WEA': {'camera': 5.0,
  'battery': 5.0,
  'fingerprint': nan,
  'screen': 5.0,
  'charger': 5.0},
 'B002UHS0UI': {'camera': 3.7,
  'battery': 3.1,
  'fingerprint': nan,
  'screen': 2.9,
  'charger': 2.8},
 'B002WTC1NG': {'camera': 3.9,
  'battery': 3.4,
  'fingerprint': nan,
  'screen': 3.0,
  'charger': 3.9},
 'B0033SFV5A': {'camera': 3.0,
  'bat

In [56]:
mean_ratings = pd.DataFrame(all_products).T

In [57]:
mean_ratings.reset_index(inplace=True)
mean_ratings

,index,camera,battery,fingerprint,screen,charger
0,B0000SX2UC,3.0,3.6,NaN,3.8,5.0
1,B000SKTZ0S,3.0,3.4,NaN,5.0,3.0
2,B001AO4OUC,3.8,2.0,NaN,2.3,3.0
3,B001DCJAJG,NaN,5.0,NaN,NaN,5.0
4,B001GQ3DJM,NaN,3.0,NaN,5.0,3.0
...,...,...,...,...,...,...
626,B07Z8BL2VW,4.0,3.8,1.0,5.0,4.3
627,B07ZDJCL76,4.3,NaN,NaN,3.0,3.0
628,B07ZHPCJW3,3.0,NaN,3.0,5.0,NaN
629,B07ZQSGP53,5.0,NaN,NaN,NaN,NaN


In [ ]:
mean_ratings.rename(columns={'index':'asin'},inplace=True)

In [ ]:
updated_mean_ratings = pd.merge(mean_ratings,new_reviews[['asin','item_title']],on='asin',how='inner')
updated_mean_ratings.drop_duplicates(subset=['asin'],keep='first',inplace=True)

In [ ]:
updated_mean_ratings.reset_index(inplace=True,drop=True)

In [ ]:
updated_mean_ratings.tail(20)

In [ ]:
## Mean ratings by features of each unique product

new_reviews.reset_index(inplace=True,drop=True)

unique_asins = new_reviews['asin'].unique()

new_reviews.loc[1,'features_and_sentiments']


all_products = {}

#for cell in new_review['features_and_sentiments']: 
all_features=set()

for product in unique_asins:
    all_products[product] = {'camera':[],'battery':[],'fingerprint':[],'screen':[],'charger':[]}
    for idx in new_reviews.index:
        if new_reviews.loc[idx,'asin'] == product:
            for feature in new_reviews.loc[idx,'features_and_sentiments']:
                all_features.add(feature[1])
                if feature[1] =='battery':
                    all_products[product]['battery'].append(feature[0])
                elif feature[1]  == 'camera':
                    all_products[product]['camera'].append(feature[0])
                elif feature[1]  == 'charger':
                    all_products[product]['charger'].append(feature[0])
                elif feature[1] == 'screen':
                    all_products[product]['screen'].append(feature[0])
                elif feature[1] == 'fingerprint':
                    all_products[product]['fingerprint'].append(feature[0])
        

for key_1,value_1 in all_products.items():
    for key_2,value_2 in all_products[key_1].items():
        try:
            all_products[key_1][key_2] = round(np.mean(all_products[key_1][key_2]),1)
        except:
            all_products[key_1][key_2] = np.nan

mean_ratings = pd.DataFrame(all_products).T

mean_ratings.reset_index(inplace=True)
mean_ratings

mean_ratings.rename(columns={'index':'asin'},inplace=True)

updated_mean_ratings = pd.merge(mean_ratings,new_reviews[['asin','item_title']],on='asin',how='inner')
updated_mean_ratings.drop_duplicates(subset=['asin'],keep='first',inplace=True)

updated_mean_ratings.reset_index(inplace=True,drop=True)

updated_mean_ratings.tail(20)

In [ ]:
for feature in new_reviews['features_and_sentiments'][5]:
    if 

In [27]:
new_reviews['summary'][5]

['parent sister husband got fancy samsung flip phone camera toy must say mine work better',
 'last time dropped back piece popped battery flew',
 'color screen nice feature easy use',
 'really us camera thier phone anyway',
 'ringtones also lack something desired decent one',
 'battery last much longer recpetion much better',
 'battery life pretty good nothing special definitely not bad']

In [22]:
samples = new_reviews.sample(100,random_state=42)

In [24]:
samples.head(10)

,asin,name,rating,date,verified,review_title,body,helpfulVotes,brand,item_title,...,originalPrice,reviews,word_count,cleaned_reviews,multi_class_sentiment,tokens,summary,sentences_with_keywords,features_and_sentiments,filter summary
13462,B077ZKKDWK,Keepin' It Real With Brett,5,"September 19, 2018",True,Exactly what I wanted in a cell phone,"The phone is the perfect size for my hand. Both the front and rear cameras take outstanding pictures. The sound quality is superb. I can get two days on a single charge and the battery saver mode is awesome. It knows I usually plug my phone in around 11 PM and it spreads the charge out overnight to my usual wake up time of 9 AM. My wife has the XA1 Ultra, this is a nice step up.",,Sony,"Sony Xperia XA2 Ultra Factory Unlocked Phone - 6"" Screen - 32GB - Silver (U.S. Warranty)",...,0.0,"Exactly what I wanted in a cell phone The phone is the perfect size for my hand. Both the front and rear cameras take outstanding pictures. The sound quality is superb. I can get two days on a single charge and the battery saver mode is awesome. It knows I usually plug my phone in around 11 PM and it spreads the charge out overnight to my usual wake up time of 9 AM. My wife has the XA1 Ultra, this is a nice step up.",86,exactly wanted cell perfect size hand front rear camera take outstanding picture sound quality superb get day single charge battery saver mode awesome know usually plug around pm spread charge overnight usual wake time wife xa ultra nice step,2,"['exactly', 'wanted', 'cell', 'perfect', 'size', 'hand', 'front', 'rear', 'camera', 'take', 'outstanding', 'picture', 'sound', 'quality', 'superb', 'get', 'day', 'single', 'charge', 'battery', 'saver', 'mode', 'awesome', 'know', 'usually', 'plug', 'around', 'pm', 'spread', 'charge', 'overnight', 'usual', 'wake', 'time', 'wife', 'xa', 'ultra', 'nice', 'step']","[get two day single charge battery saver mode awesome, front rear camera take outstanding picture]","[both the front and rear cameras take outstanding pictures., i can get two days on a single charge and the battery saver mode is awesome.]","[(5, camera), (5, battery)]",True
17851,B07L78G3D2,carol,4,"November 20, 2019",True,Very good product,Battery takes a good charge quickly but not much use time.may need to upgrade battery..otherwise great phone,1.0,Nokia,"Nokia 3 - Android 9.0 Pie - 16 GB - Unlocked Smartphone (AT&T/T-Mobile/Metropcs/Cricket/Mint) - 5.0"" HD Screen - Copper",...,0.0,Very good product Battery takes a good charge quickly but not much use time.may need to upgrade battery..otherwise great phone,20,good product battery take good charge quickly much use time may need upgrade battery otherwise great,2,"['good', 'product', 'battery', 'take', 'good', 'charge', 'quickly', 'much', 'use', 'time', 'may', 'need', 'upgrade', 'battery', 'otherwise', 'great']",[good product battery take good charge quickly not much use time may need upgrade battery otherwise great phone],[very good product battery takes a good charge quickly but not much use time.may need to upgrade battery..otherwise great phone],"[(5, battery)]",True
2645,B00F2SKPIM,jones,5,"January 11, 2014",True,LOVE my GN3,I LOVE my GN3! The screen is huge and beautiful! So clear. I've only had it a couple of weeks so still have much to learn. No regrets!,1.0,Samsung,"Samsung Galaxy Note 3, Black 32GB (Verizon Wireless)",...,0.0,LOVE my GN3 I LOVE my GN3! The screen is huge and beautiful! So clear. I've only had it a couple of weeks so still have much to learn. No regrets!,31,love gn love gn screen huge beautiful clear couple week still much learn regret,2,"['love', 'gn', 'love', 'gn', 'screen', 'huge', 'beautiful', 'clear', 'couple', 'week', 'still', 'much', 'learn', 'regret']",[screen huge beautiful],[the screen is huge and beautiful!],"[(5, screen)]",True
2255,B00E6FGSHY,John B,4,"March 8, 2016",True,Phone cam in excellent condition - if it had a screen replacement done ...,Phone cam in excellent condition - if it had a screen replace

In [ ]:
samples.to_csv('../data/samples_for_evaluation.csv',index=False)

In [ ]:
#new_reviews.to_csv('../data/cleaned_combined_data_with_keywords.csv',index=False)

In [ ]:
iphone_xs = new_reviews[new_reviews['asin'] == 'B07RT1X4FJ']

In [ ]:
pd.set_option('display.max_colwidth',None)
iphone_xs['reviews']

In [ ]:
iphone_xs['summary'][63325]

In [ ]:
pd.set_option('display.max_colwidth',None)
iphone_xs['features_and_sentiments']

In [ ]:
iphone_xs.loc[63202,'features_and_sentiments']

In [ ]:
def multi_class_sentiment(rating):
    if rating >= 4:
        return 2 #positive 
    elif rating == 3:
        return 1 #neutral
    else:
        return 0 #negative
    

new_reviews['multi_class_sentiment'] = new_reviews['rating'].map(multi_class_sentiment)

## With BERT

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
new_reviews.head(2)

In [ ]:
new_reviews.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
new_reviews['multi_class_sentiment'].value_counts(normalize=True)

In [ ]:
len(new_reviews['rating'].unique())

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(new_reviews.index.values,
                                                  new_reviews.multi_class_sentiment.values,
                                                  test_size = 0.25,
                                                  random_state= 42,
                                                  stratify=new_reviews.multi_class_sentiment.values)

In [ ]:
y_train.shape

In [ ]:
y_val.shape

In [ ]:
new_reviews['data_type'] = ['not_set']*new_reviews.shape[0]

In [ ]:
new_reviews.loc[X_train, 'data_type'] = 'train'
new_reviews.loc[X_val, 'data_type'] = 'val'

In [ ]:
new_reviews.head(2)

In [ ]:
new_reviews.groupby(['multi_class_sentiment', 'data_type']).count()

## Loading Tokenizer and

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
new_reviews.columns

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    new_reviews[new_reviews.data_type=='train'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    new_reviews[new_reviews.data_type=='val'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(new_reviews[new_reviews.data_type=='train'].multi_class_sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val= torch.tensor(new_reviews[new_reviews.data_type=='val'].multi_class_sentiment.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

In [ ]:
len(dataset_val)

## Setting up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)


## Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
dataloader_train 

In [ ]:
16530/32

In [ ]:
5511/32

## Setting Up Optimiser and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Defining our Performance Metrics

In [ ]:
def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    correct_pred = 0
    total_count = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        
        correct_pred = correct_pred + len(y_preds[y_preds==label])
        total_count = total_count + len(y_true)
        
    print(f'Total Accuracy:{correct_pred/total_count}' )

## Creating Training Loop

In [ ]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('../data/finetuned_BERT_epoch_1_3classes.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

## Loading Tokenizer and Encoding Data by Sentences

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
new_reviews.sentences_with_keywords.values[3833]

In [ ]:
summary = set()
    
encoded_data_features = tokenizer.batch_encode_plus(
    new_reviews.sentences_with_keywords.values[3833], 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_features = encoded_data_features['input_ids']
attention_masks_features = encoded_data_features['attention_mask']
    #labels_features = torch.tensor(df[df.data_type=='val'].label.values)

dataset_features = TensorDataset(input_ids_features, attention_masks_features)

dataloader_features = DataLoader(dataset_features , 
                                sampler=SequentialSampler(dataset_features ), 
                                batch_size=batch_size)

In [ ]:
count=0
for batch in dataloader_features:
    count += 1

print(count)
    

In [ ]:
model.eval()
    
predictions = []

for batch in dataloader_features:

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                     }

    with torch.no_grad():        
            outputs = model(**inputs)
        
        
    
predict_v2 = torch.argmax(outputs[0],dim=1)

    
rating_score = predict_v2

rating_score

In [ ]:
a = [[1,2,3],[4,5,6]]

np.concatenate(a,axis=0)

In [ ]:
model.eval()
predictions = []
    
for i,batch in enumerate(dataloader_validation):
    
    if i <= 1:
    
        
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            }

        with torch.no_grad():        
            outputs = model(**inputs)

        # since we have no loss, the only thing returned is logits
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        #print(len(predictions))
predictions = np.concatenate(predictions, axis=0)
#print(predictions)
# get highest prob dimension as prediction
preds_flat = np.argmax(predictions, axis=1)
print(len(preds_flat))
print(preds_flat.shape)
preds_flat


In [ ]:
model.eval()
predictions = []
    
for i,batch in enumerate(dataloader_validation):
    
    if i <= 1:
    
        
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            }

        with torch.no_grad():        
            outputs = model(**inputs)

        # since we have no loss, the only thing returned is logits
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        #print(len(predictions))
#predictions = np.concatenate(predictions, axis=0)
#print(predictions)
# get highest prob dimension as prediction
preds_flat = np.argmax(predictions, axis=1).flatten()
print(len(preds_flat))
print(preds_flat.shape)
preds_flat

In [ ]:
preds_flat.shape

In [ ]:
predictions[0]

In [ ]:
model.eval()
predictions = []
    
for i,batch in enumerate(dataloader_validation):
    
    if i <= 1:
    
        
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            }

        with torch.no_grad():        
            outputs = model(**inputs)

        # since we have no loss, the only thing returned is logits
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        print(len(predictions))
predictions = np.concatenate(predictions, axis=0)

# get highest prob dimension as prediction
preds_flat = np.argmax(predictions, axis=1).flatten() 
len(preds_flat)
preds_flat

In [ ]:
model.eval()
    
predictions = []

for batch in dataloader_features:

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                     }

with torch.no_grad():        
        outputs = model(**inputs)
        
        
    
predict_v2 = torch.argmax(outputs[0],dim=1)

    
rating_score = predict_v2

rating_score

In [ ]:
predicted_ratings = []

for score in rating_score:
    if float(score) == 2.0:
        rating = 5
        predicted_ratings.append(rating)
    elif float(score) == 1.0:
        rating = 3
        predicted_ratings.append(rating)
    else:
        rating = 1
        predicted_ratings.append(rating)

predicted_ratings

In [ ]:
def bert_sentiments (summarised_reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    
    summary = set()
    
    encoded_data_features = tokenizer.batch_encode_plus(
    summarised_reviews, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

    input_ids_features = encoded_data_features['input_ids']
    attention_masks_features = encoded_data_features['attention_mask']
    #labels_features = torch.tensor(df[df.data_type=='val'].label.values)

    dataset_features = TensorDataset(input_ids_features, attention_masks_features)

    dataloader_features = DataLoader(dataset_features , 
                                       sampler=SequentialSampler(dataset_features ), 
                                       batch_size=batch_size)

    
    model.eval()
    
    predictions = []

    for batch in dataloader_features:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                     }

    with torch.no_grad():        
        outputs = model(**inputs)
        
        
    
    predict_v2 = torch.argmax(outputs[0],dim=1)

    
    rating_score = predict_v2
    
    predicted_ratings = []

    for score in rating_score:
        if float(score) == 2.0:
            rating = 5
            predicted_ratings.append(rating)
        elif float(score) == 1.0:
            rating = 3
            predicted_ratings.append(rating)
        else:
            rating = 1
            predicted_ratings.append(rating)
    
    for i,cleaned_sentence in enumerate(summarised_reviews):        
        for word in list_of_keywords:
            if word in cleaned_sentence:
                summary.add((float(predicted_ratings[i]),word))
                
    
    return summary



bert_sentiments(new_reviews.sentences_with_keywords.values[5])   

In [ ]:
test_row = bert_sentiments(new_reviews.sentences_with_keywords.values[5])  

In [ ]:
count = 0
for i,batch in enumerate(dataloader_features):
    print(len(batch[1]))
    batch = tuple(b.to(device) for b in batch)
    count += 1

#print(batch)
print(count)

In [ ]:
count = 0
for i,batch in enumerate(dataloader_validation):
    print(len(batch[2]))
    batch = tuple(b.to(device) for b in batch)
    count += 1

#print(batch)
print(count)

In [ ]:
iphone_xs['bert_analysis'] = iphone_xs['sentences_with_keywords'].apply(bert_sentiments)

In [ ]:
iphone_xs.shape

In [ ]:
new_reviews['sentences_with_keywords'][:10].apply(bert_sentiments)

In [ ]:
iphone_xs['reviews'][:5]

In [ ]:
iphone_xs['sentences_with_keywords'][:5]

In [ ]:
iphone_xs['features_and_sentiments'][:5] #vader

In [ ]:
iphone_xs['bert_analysis'][:5] #bert

In [ ]:
iphone_xs['logreg_pred'][:5] #logreg

In [ ]:
import pickle
logreg_model = pickle.load(open('../data/logreg_3classes.pkl', 'rb'))

In [ ]:
def logreg_classification (reviews):
    list_of_keywords = ['camera','screen','battery','simcard','touchscreen','fingerprint','fingerprints',
                        'ringtones','charger']
    summary = set()
    pred = logreg_model.predict(reviews)
    
    predicted_ratings = []

    for score in pred:
        if float(score) == 2.0:
            rating = 5.0
            predicted_ratings.append(rating)
        elif float(score) == 1.0:
            rating = 3.0
            predicted_ratings.append(rating)
        else:
            rating = 1.0
            predicted_ratings.append(rating)
    
    for i,cleaned_sentence in enumerate(reviews):        
        for word in list_of_keywords:
            if word in cleaned_sentence:
                summary.add((predicted_ratings[i],word))
    
    
                
    
    return summary

In [ ]:
iphone_xs['logreg_pred'] = iphone_xs['summary'].apply(logreg_classification)